In [9]:
import sys
import csv
import numpy as np
import matplotlib.pyplot as plt
import os
# print(os.getcwd())

In [10]:
# reference sound pressure level for airborne sound is 20 micropascals (μPa or e-6)

# https://media.hbkworld.com/m/ac047760baf884bf/original/Sound-Intensity.pdf

db_spl_to_pascal = lambda db: np.power( 10, db/20 ) * 20e-6

pascal_to_db_spl = lambda pa: 20 * np.log10( pa / 20e-6 )  


In [11]:
def timebase_to_sampling_period(timebase: int) -> float:
    """
    Calculates the sampling period (s) based on the timebase

    when the timebase (n) is 9, the picoscope will sample every (9+1) * 10e-7 s = 1 us. 
    when the timebase (n) is 0, the picoscope will sample every (0+1) * 10e-7 s = 0.1 us. 
    """
    return (timebase+1) * 1e-7



def free_field_correction( pressure_rms, correction=0.78 ):
    """
    Free-field correction curves for Bruel & Kjaer Type 4138 from the spec:  

    https://kiptm.ru/images/Production/bruel/table_pdf/microphones_preamps/2017.02/4138.pdf    

    Free-field corrections must be added to the pressure (actuator) response of the microphone in
    order to obtain the free-field response at a particular angle of incidence. 
    
    The free-field correction represents the increase of sound pressure caused by diffraction of the sound waves
    around the microphone and is only significant at very high frequencies where the wavelength
    is comparable with the external dimensions of the microphone.

    The correction value of 0.78 dB for an angle of incidence q = 90 degrees, was obtained with a graphic scanner
    (https://automeris.io/WebPlotDigitizer/) and the graph in the spec sheet above 

    """

    # peak-to-peak amplitude represents the full range between the positive and negative peaks of the waveform.
    # to approximate the RMS value, we divide this peak-to-peak amplitude by the square root of 2

    return db_spl_to_pascal( pascal_to_db_spl(pressure_rms) - correction )

def remove_dc_bias(voltages_peak_to_peak):

    # # remove DC bias
    # if np.max(voltages_peak_to_peak) > np.min(voltages_peak_to_peak):
    #     voltages_peak_to_peak -= dc_bias
    # else: 
    #     voltages_peak_to_peak += dc_bias
    pass


def pressure_from_voltage_peak_to_peak( voltage_peak_to_peak, preamplifier_gain=3.16/1000 ): 
    """
    Calculate pressure Root Mean Square (RMS) from a list of voltage measurements with the BEAST.

    RMS = sqrt( 1/N * sum(p(n)^2) ) 
    
    Args:
        voltages: list of voltages peak-to-peak [V] sampled for a given spatial coordinate 

        preamplifier_gain: user-configured gain [mV/Pa] (e.g. 1mV, 3.16mV)
        
    Returns:
        Pressure RMS for the spatial coordinate
    """

    pressure = voltage_peak_to_peak / preamplifier_gain
    
    return pressure


def pressure_rms_from_voltages_peak_to_peak( voltages_peak_to_peak, preamplifier_gain=3.16/1000 ): 
    """
    Calculate pressure Root Mean Square (RMS) from a list of voltage measurements with the BEAST.

    RMS = sqrt( 1/N * sum(p(n)^2) ) 
    
    Args:
        voltages: list of voltages peak-to-peak [V] sampled for a given spatial coordinate 

        preamplifier_gain: user-configured gain [mV/Pa] (e.g. 1mV, 3.16mV)
        
    Returns:
        Pressure RMS for the spatial coordinate
    """
    voltage_rms = np.sqrt(np.mean(np.square(voltages_peak_to_peak)))
    
    pressure_rms = voltage_rms / preamplifier_gain
    
    return pressure_rms


def phase_from_voltages_peak_to_peak( voltages_peak_to_peak, scope_sampling_period ):
    """
    Calculates the phase angle from a list of voltage measurements, 
    
    waveform based on the minimum voltage point within a period of the waveform.

    Depends on sampling period and the number of samples per segment. 
    
    e.g. 
    
    if theses values are 1000 and 0.1 us, the period of each segment is 1000 * 0.1 us = 0.1 ms  
    Thus we get 40 cycles at 40 kHz. 
    
    Args:
        voltage_list: List of voltage peak-to-peak measurements for this data point.

        scope_sampling_period: time in microseconds between voltage samples. 
                                This is dependendent of the timebase set on the Beast GUI.
                                e.g. when the timebase (n) is 9, the picoscope will then sample every (9+1)*10 -7 s = 1 us. 

        samples_per_segment: the number of samples for each segment or point in the field 
        
    Returns:
        Phase measured for a given segment.
        
    """
    
    wave_period = 1/40000                                     # period of a 40 kHz wave, the time taken to complete one cycle, set to .25 us   
    
    samples_per_period = wave_period / scope_sampling_period  # e.g. 25 us / 0.1 us = 250 samples per period

    min_voltage = min(voltages_peak_to_peak)

    # index of minimum voltage adjusted to zero, assuming that the waveform starts at index 0
    min_voltage_zero_index = list(voltages_peak_to_peak).index(min_voltage) - 1

    # mod ensures that the index remains within the range of a single period.
    # we convert the index to an angle in radians by scaling the value to cover 2π radians within a single period
    # we normalizes the angle within the range of 0 to 2π radians, by dividing the scaled value by samples_per_period
    # this ensures that the phase angle is proportional to the relative position of the minimum voltage within a period
    # subtracting π from the normalized angle adjusts the phase angle to be centered around zero [-π, +π] radians
    return 2*np.pi * np.mod( min_voltage_zero_index, samples_per_period ) / samples_per_period - np.pi



In [69]:
class VADProcessor:

    def __init__(self):
        self.x_size = -1 
        self.y_size = -1
        self.z_size = -1
        self.step_size = -1
        self.samples_per_segment = -1
        self.sampling_period = -1
        self.preamplifier_gain = -1
        self.free_field_correction_factor = -1

        self.coords = []
        self.voltages = []
        self.amplitudes = []
        self.phases = []

        self.complex_pressure_field = {}
        return 

    def set_free_field_correction( self, free_field_correction=0.78 ):
        self.free_field_correction_factor = free_field_correction
        return

    def set_preamplifier_gain( self, preamplifier_gain=3.16/1000 ):
        self.preamplifier_gain = preamplifier_gain
        return

    def set_sampling_period( self, sampling_period=0.250 ):
        self.sampling_period = sampling_period
        return

    def parse_header( self, reader ) -> None:
        """
        Parses the Beast CSV file header with pressure measurements.
        
        It expects this header structure on the first 8 lines

        1 Scan Date	Scan Time			
        2 02-Jun	13:17
        3 X size mm	Y size mm	Z size mm	Step size mm	
        4 0	100	120	1	
        5 Segments	Samples/Segment	TimeBase	Sample Interval	RMS
        6 1	1000	0	100	No
        7 Array X	Array Y	Array Z		
        8 1	101	121		
        
        Args:
            reader – the CSV reader

        Returns 
            x_size, y_size, z_size, step_size

        """

        for _ in range(3):  # skip date + size's header
            next(reader)
        
        line = next(reader, None)
        self.step_size = float(line[3])

        next(reader, None)
        line = next(reader, None)
        self.samples_per_segment, self.sample_interval = int(line[1]), int(line[3]) 

        next(reader, None)
        line = next(reader, None)
        self.x_size, self.y_size, self.z_size = int(float(line[0])), int(float(line[1])), int(float(line[2]))
        
        return 


    def parse_voltage_samples( self, reader ) -> None:
        """
        Parses 
        """
        for i, line in enumerate(reader):
            if i % 2 == 0: 
                self.coords.append( ( float(line[0]), float(line[1]), float(line[2]) ) )
            else:
                self.voltages.append( [ float(voltage) for voltage in line ] ) # list of lists of N samples
        return



    def parse_file(self, filename) -> None:
        
        with open(filename, 'r') as file:
            
            self.coords = []
            self.voltages = []
            self.amplitudes = []
            self.phases = []

            reader = csv.reader(file)

            # Parse the first 8 header lines
            self.parse_header(reader)    

            # Parse point coordinates and respective voltage samples
            self.parse_voltage_samples(reader)

            j, reverse_scan_counter, amplitudes, phases = 0, 0, [], []
            
            for i, voltage_samples in enumerate(self.voltages):
                j += 1 
                pressure_rms_corrected = free_field_correction(
                                                pressure_rms_from_voltages_peak_to_peak( voltage_samples, self.preamplifier_gain ), # preamplifier_gain = 3.16/1000
                                                correction = self.free_field_correction_factor # correction = 0.78
                                            )
                
                amplitudes.append(pressure_rms_corrected)
                
                phase_normalised = phase_from_voltages_peak_to_peak( voltage_samples, self.sampling_period ) 
                phases.append(phase_normalised)

                if j % int(self.y_size) == 0:
                    reverse_scan_counter +=1          
                    if reverse_scan_counter % 2 == 0 and amplitudes != []:
                        amplitudes.reverse()
                        phases.reverse()

                    self.amplitudes.append(amplitudes)  
                    self.phases.append(phases)
                    amplitudes, phases = [], []
                
                # complex_pressure[ tuple(self.coords[i]) ]= ( pressure_rms_corrected, phase_normalised )
                
                # self.complex_pressure_field[ tuple(self.coords[i]) ] = ( pressure_rms_corrected, phase_normalised  )
            self.complex_pressure_field[ tuple(self.coords[i]) ] = ( self.amplitudes )

        return 


    def plot_amplitude( self ):
        
        # plot = np.array( list(self.complex_pressure_field.values()),

        sizes = list(filter(lambda x: x > 1, [ self.x_size, self.y_size, self.z_size ]))


        amplitude_plot = np.array( 
                            list(self.amplitudes),
                            dtype="float"
                            ).reshape( sizes[0], sizes[1] )

        # plt.axhline(100, c="magenta", ls="--", label='FP Z=0.100 m')
        
        # plt.axhline(
        #     np.squeeze( (np.where(self.amplitudes == np.amax(self.amplitudes)))[0] ),
        #                 c="red", 
        #                 label=f'PP Z=0.051 m ({ np.amax(self.amplitudes) })' 
        #     )

        plt.imshow(amplitude_plot)
        ax = plt.gca()
        ax.set_ylim(ax.get_ylim()[::-1])
        plt.legend(loc='lower left', bbox_to_anchor=(1, 0.5))
        plt.show()

        return  

    def plot( self ):

        sizes = list(filter(lambda x: x > 1, [ self.x_size, self.y_size, self.z_size ]))

        amplitude_plot = np.array( list(self.amplitudes), dtype="float" ).reshape( sizes[0], self.x_size ) 

        phase_plot = np.array( list(self.phases), dtype="float" ).reshape( sizes[0], self.x_size ) 
        
        fig, (ax1, ax2) = plt.subplots(1,2, figsize=(10, 5))
        im1 = ax1.imshow(amplitude_plot, plt.get_cmap("jet"), origin="lower")
        im2 = ax2.imshow(phase_plot, origin="lower", vmax=3)




In [71]:
processor = VADProcessor()
# processor.set_free_field_correction(7.8)
processor.set_free_field_correction(4.2)
processor.set_preamplifier_gain(3.16/1000)
processor.set_sampling_period(timebase_to_sampling_period(0)) # setting a timebase of 0 yields a 0.1 us sampling period
# processor.parse_file(f'./data/example.csv')

processor.parse_file(f'./data/20231027-61-hex-PAT-channel-3-pha-0-amp-1.0-Z-0-31-3.csv')
# processor.plot()

print("Voltage_RMS: {}".format( np.sqrt(np.mean(np.square(processor.voltages[0]))) ) )
print("Pressure_RMS: {}".format( pressure_rms_from_voltages_peak_to_peak( processor.voltages, 3.16/1000 )) )
print("Pressure_RMS_corrected_At_4.2: {}".format( free_field_correction(np.sqrt(np.mean(np.square(processor.voltages)))/(3.16/1000), 4.2) ) )
# print("Points: {} x {}  ".format( processor.x_size, processor.z_size ) )
print("Peak pressure: {} ".format( processor.amplitudes ) )
print("Phases: {} ".format( processor.phases ) )
print("")

processor.parse_file(f'./data/20231027-61-hex-PAT-channel-3-pha-0-amp-1.0-Z-0-31-2.csv')
# processor.plot()


print("Voltage_RMS: {}".format( np.sqrt(np.mean(np.square(processor.voltages[0]))) ) )
print("Pressure_RMS: {}".format( pressure_rms_from_voltages_peak_to_peak( processor.voltages, 3.16/1000 )) )
print("Pressure_RMS_corrected_At_4.2: {}".format( free_field_correction(np.sqrt(np.mean(np.square(processor.voltages)))/(3.16/1000), 4.2) ) )
# print("Points: {} x {}  ".format( processor.x_size, processor.z_size ) )
print("Peak pressure: {} ".format( processor.amplitudes ) )
print("Phases: {} ".format( processor.phases ) )
print("")

processor.parse_file(f'./data/20231027-61-hex-PAT-channel-3-pha-0-amp-1.0-Z-0-31-1.csv')
# processor.plot()


print("Voltage_RMS: {}".format( np.sqrt(np.mean(np.square(processor.voltages[0]))) ) )
print("Pressure_RMS: {}".format( pressure_rms_from_voltages_peak_to_peak( processor.voltages, 3.16/1000 )) )
print("Pressure_RMS_corrected_At_4.2: {}".format( free_field_correction(np.sqrt(np.mean(np.square(processor.voltages)))/(3.16/1000), 4.2) ) )
# print("Points: {} x {}  ".format( processor.x_size, processor.z_size ) )
print("Peak pressure: {} ".format( processor.amplitudes ) )
print("Phases: {} ".format( processor.phases ) )
print("")

processor.parse_file(f'./data/20231027-61-hex-PAT-channel-3-pha-0-amp-1.0-Z-0-31-0.csv')
# processor.plot()


print("Voltage_RMS: {}".format( np.sqrt(np.mean(np.square(processor.voltages[0]))) ) )
print("Pressure_RMS: {}".format( pressure_rms_from_voltages_peak_to_peak( processor.voltages, 3.16/1000 )) )
print("Pressure_RMS_corrected_At_4.2: {}".format( free_field_correction(np.sqrt(np.mean(np.square(processor.voltages)))/(3.16/1000), 4.2) ) )
# print("Points: {} x {}  ".format( processor.x_size, processor.z_size ) )
print("Peak pressure: {} ".format( processor.amplitudes ) )
print("Phases: {} ".format( processor.phases ) )
print("")



Voltage_RMS: 4.425984000253651
Pressure_RMS: 580.616982285796
Pressure_RMS_corrected_At_4.2: 358.0055292733192
Peak pressure: [[863.6201306567392, 727.8615534949662, 671.3941829066031, 668.3490943885571, 620.0245800072237, 516.1561683235163, 436.7436564614859, 401.60997766355786, 383.5183717236893, 344.50265245678713, 294.8666466572213, 265.2905264063655, 255.90384832269072, 244.74074506908863, 219.70453390148586, 197.6822961557283, 188.218169738348, 183.17003747171378, 173.02616446387495, 158.80196386648717, 148.62937785477553, 144.90619815461753, 140.59832773545847, 130.52720004170052, 119.78334399561777, 114.12537844067487, 111.29270178581704, 107.46082230757382, 100.17595328239106, 93.35810134242621, 89.15723397415942, 86.87849280831088]] 
Phases: [[1.156106096521043, 1.4828317324943807, 1.9352210746113103, 2.4630086404143974, 3.1415926535897922, -2.4378758991856797, -1.7592918860102857, -1.1309733552923258, -0.4775220833456504, 0.25132741228718114, 0.9550441666912954, 1.6336281798

In [83]:
processor = VADProcessor()
# processor.set_free_field_correction(7.8)
processor.set_free_field_correction(4.2)
processor.set_preamplifier_gain(3.16/1000)
processor.set_sampling_period(timebase_to_sampling_period(0)) # setting a timebase of 0 yields a 0.1 us sampling period
# processor.parse_file(f'./data/example.csv')

processor.parse_file(f'./data/20231027-61-hex-PAT-channel-3-pha-0-amp-1.0-0.csv')

print("Voltage_RMS: {}".format( np.sqrt(np.mean(np.square(processor.voltages[0]))) ) )
print("Pressure_RMS: {}".format( pressure_rms_from_voltages_peak_to_peak( processor.voltages, 3.16/1000 )) )
print("Pressure_RMS_corrected_At_4.2: {}".format( free_field_correction(np.sqrt(np.mean(np.square(processor.voltages)))/(3.16/1000), 4.2) ) )
print("Points: {} x {}  ".format( processor.x_size, processor.z_size ) )
print("Peak pressure: {} ".format( processor.amplitudes ) )
print("Phases: {} ".format( processor.phases ) )

Voltage_RMS: 7.834272589128921
Pressure_RMS: 2205.3182381323504
Pressure_RMS_corrected_At_4.2: 1359.7882031463803
Points: 1 x 1  
Peak pressure: [[1528.6624435687734, 1166.7214956683003]] 
Phases: [[0.5026548245743663, 1.231504320207197]] 


In [84]:
processor = VADProcessor()
# processor.set_free_field_correction(7.8)
processor.set_free_field_correction(4.2)
processor.set_preamplifier_gain(3.16/1000)
processor.set_sampling_period(timebase_to_sampling_period(0)) # setting a timebase of 0 yields a 0.1 us sampling period
# processor.parse_file(f'./data/example.csv')

processor.parse_file(f'./data/20231027-61-hex-PAT-channel-3-pha--1.15-amp-1.0-Z-0-31-0.csv')
print("Voltage_RMS: {}".format( np.sqrt(np.mean(np.square(processor.voltages[0]))) ) )
print("Pressure_RMS: {}".format( pressure_rms_from_voltages_peak_to_peak( processor.voltages, 3.16/1000 )) )
print("Pressure_RMS_corrected_At_4.2: {}".format( free_field_correction(np.sqrt(np.mean(np.square(processor.voltages)))/(3.16/1000), 4.2) ) )
# print("Points: {} x {}  ".format( processor.x_size, processor.z_size ) )
print("Peak pressure: {} ".format( processor.amplitudes ) )
print("Phases: {} ".format( processor.phases ) )


Voltage_RMS: 4.357522055527079
Pressure_RMS: 574.8932674333855
Pressure_RMS_corrected_At_4.2: 354.4763153032418
Peak pressure: [[850.2614936516366, 720.6488248968152, 666.3548645067607, 661.5424683729008, 610.6037760499687, 512.1238022199132, 433.7381642552732, 397.63327495619916, 381.485277739926, 342.5699772900002, 292.35561913437886, 261.9647808549446, 253.43439462799995, 243.60695495298825, 219.15143764207264, 196.50999387937037, 186.95052516318856, 182.70173425220938, 172.79456949774837, 157.82813866272107, 147.53507364549293, 145.0046979509666, 141.63266798371603, 131.16598384512508, 119.38802134399575, 113.21586310849541, 111.02790567270243, 107.15786583995269, 99.38247289689976, 92.769581001412, 89.76526615037008, 88.53398535343392]] 
Phases: [[-2.563539605329271, -2.2619467105846534, -1.8095573684677224, -1.2566370614359181, -0.6031857894892414, 0.12566370614359101, 0.7539822368615474, 1.3823007675795074, 2.0357520395261846, 2.789734276387734, -2.8148670176164567, -2.111150263

In [85]:
processor = VADProcessor()
# processor.set_free_field_correction(7.8)
processor.set_free_field_correction(4.2)
processor.set_preamplifier_gain(3.16/1000)
processor.set_sampling_period(timebase_to_sampling_period(0)) # setting a timebase of 0 yields a 0.1 us sampling period
# processor.parse_file(f'./data/example.csv')

processor.parse_file(f'./data/20231027-61-hex-PAT-channel-3-pha-1.15-amp-1.0-Z-0-31-0.csv')
print("Voltage_RMS: {}".format( np.sqrt(np.mean(np.square(processor.voltages[0]))) ) )
print("Pressure_RMS: {}".format( pressure_rms_from_voltages_peak_to_peak( processor.voltages, 3.16/1000 )) )
print("Pressure_RMS_corrected_At_4.2: {}".format( free_field_correction(np.sqrt(np.mean(np.square(processor.voltages)))/(3.16/1000), 4.2) ) )
# print("Points: {} x {}  ".format( processor.x_size, processor.z_size ) )
print("Peak pressure: {} ".format( processor.amplitudes ) )
print("Phases: {} ".format( processor.phases ) )

Voltage_RMS: 4.367488674287207
Pressure_RMS: 576.0884529517372
Pressure_RMS_corrected_At_4.2: 355.21326072015535
Peak pressure: [[852.2062301430792, 724.3189098457424, 665.0922425781783, 661.5681283267998, 617.4936126850292, 512.3672911268598, 433.58928939066516, 398.75419063884465, 381.1431428228415, 342.90712289175843, 292.51214887236, 262.1818294046962, 253.15449878368733, 243.4286036507672, 219.2374039071527, 196.65291655730212, 186.8410466882732, 182.6703643205897, 173.01518397905448, 157.8464181595725, 147.45706880759238, 145.20387601535003, 141.99043393369675, 131.2418234416319, 119.4476697080035, 113.28253795647475, 111.06712525781137, 106.98884880010534, 99.224503718781, 92.86480314540543, 89.9577593840405, 88.7906347162921]] 
Phases: [[1.709026403552845, 2.0357520395261854, 2.4630086404143974, 2.9907962062174818, -2.6138050877867096, -1.8849555921538783, -1.2315043202072014, -0.6283185307179595, 0.025132741228718114, 0.7539822368615487, 1.4576989912656622, 2.136283004441059, 

In [97]:
processor = VADProcessor()
# processor.set_free_field_correction(7.8)
processor.set_free_field_correction(4.2)
processor.set_preamplifier_gain(3.16/1000)
processor.set_sampling_period(timebase_to_sampling_period(0)) # setting a timebase of 0 yields a 0.1 us sampling period
# processor.parse_file(f'./data/example.csv')

processor.parse_file(f'./data/20231027-61-hex-PAT-channel-3-pha-0-amp-1.0-Z-0-31-0.csv')
print("Voltage_RMS: {}".format( np.sqrt(np.mean(np.square(processor.voltages[0]))) ) )
print("Pressure_RMS: {}".format( pressure_rms_from_voltages_peak_to_peak( processor.voltages, 3.16/1000 )) )
print("Pressure_RMS_corrected_At_4.2: {}".format( free_field_correction(np.sqrt(np.mean(np.square(processor.voltages)))/(3.16/1000), 4.2) ) )
# print("Points: {} x {}  ".format( processor.x_size, processor.z_size ) )
print("Peak pressure: {} ".format( processor.amplitudes ) )
print("Phases: {} ".format( processor.phases ) )

processor.parse_file(f'./data/20231027-61-hex-PAT-channel-3-pha-0-amp-1.0-Z-0-31-1.csv')
print("Voltage_RMS: {}".format( np.sqrt(np.mean(np.square(processor.voltages[0]))) ) )
print("Pressure_RMS: {}".format( pressure_rms_from_voltages_peak_to_peak( processor.voltages, 3.16/1000 )) )
print("Pressure_RMS_corrected_At_4.2: {}".format( free_field_correction(np.sqrt(np.mean(np.square(processor.voltages)))/(3.16/1000), 4.2) ) )
# print("Points: {} x {}  ".format( processor.x_size, processor.z_size ) )
print("Peak pressure: {} ".format( processor.amplitudes ) )
print("Phases: {} ".format( processor.phases ) )

processor.parse_file(f'./data/20231027-61-hex-PAT-channel-3-pha-0-amp-1.0-Z-0-31-2.csv')
print("Voltage_RMS: {}".format( np.sqrt(np.mean(np.square(processor.voltages[0]))) ) )
print("Pressure_RMS: {}".format( pressure_rms_from_voltages_peak_to_peak( processor.voltages, 3.16/1000 )) )
print("Pressure_RMS_corrected_At_4.2: {}".format( free_field_correction(np.sqrt(np.mean(np.square(processor.voltages)))/(3.16/1000), 4.2) ) )
# print("Points: {} x {}  ".format( processor.x_size, processor.z_size ) )
print("Peak pressure: {} ".format( processor.amplitudes ) )
print("Phases: {} ".format( processor.phases ) )



Voltage_RMS: 4.267667249668802
Pressure_RMS: 567.7936641629468
Pressure_RMS_corrected_At_4.2: 350.09873541149085
Peak pressure: [[832.7285746056086, 701.1944838060454, 648.6866450767499, 649.2930829074488, 612.8107934766002, 513.0095618732547, 431.7144987164783, 393.7123584911536, 376.3348792031721, 341.30620505049393, 293.6532144259775, 263.5540127089719, 253.28201617955526, 242.42381415770566, 219.38381001771918, 198.66202889861478, 188.72801118087784, 183.6053944998784, 172.4471492662304, 157.70546139328093, 147.42451982661203, 144.4871940861324, 139.85482399187805, 130.01643627011933, 120.34939741468645, 115.4714683132646, 112.53672830842208, 106.77850924475202, 98.96609997738805, 94.25072523040168, 92.85644444305197, 89.99475164624023]] 
Phases: [[1.2566370614359164, 1.5330972149518178, 1.9603538158400289, 2.4630086404143956, -3.141592653589793, -2.43787589918568, -1.784424627239004, -1.1812388377497645, -0.502654824574368, 0.20106192982974402, 0.9047786842338583, 1.60849543863797

In [111]:
processor = VADProcessor()
# processor.set_free_field_correction(7.8)
processor.set_free_field_correction(4.2)
processor.set_preamplifier_gain(3.16/1000)
processor.set_sampling_period(timebase_to_sampling_period(0)) # setting a timebase of 0 yields a 0.1 us sampling period
# processor.parse_file(f'./data/example.csv')

processor.parse_file(f'./data/20231027-61-hex-PAT-channel-3-pha--1-amp-1.0-Z-0-31-0.csv')
print("Voltage_RMS: {}".format( np.sqrt(np.mean(np.square(processor.voltages[0]))) ) )
print("Pressure_RMS: {}".format( pressure_rms_from_voltages_peak_to_peak( processor.voltages, 3.16/1000 )) )
print("Pressure_RMS_corrected_At_4.2: {}".format( free_field_correction(np.sqrt(np.mean(np.square(processor.voltages)))/(3.16/1000), 4.2) ) )
# print("Points: {} x {}  ".format( processor.x_size, processor.z_size ) )
print("Peak pressure: {} ".format( processor.amplitudes ) )
print("Phases: {} ".format( processor.phases ) )
print("")

processor.parse_file(f'./data/20231027-61-hex-PAT-channel-3-pha--1-amp-1.0-Z-0-31-1.csv')
print("Voltage_RMS: {}".format( np.sqrt(np.mean(np.square(processor.voltages[0]))) ) )
print("Pressure_RMS: {}".format( pressure_rms_from_voltages_peak_to_peak( processor.voltages, 3.16/1000 )) )
print("Pressure_RMS_corrected_At_4.2: {}".format( free_field_correction(np.sqrt(np.mean(np.square(processor.voltages)))/(3.16/1000), 4.2) ) )
# print("Points: {} x {}  ".format( processor.x_size, processor.z_size ) )
print("Peak pressure: {} ".format( processor.amplitudes ) )
print("Phases: {} ".format( processor.phases ) )
print("")

processor.parse_file(f'./data/20231027-61-hex-PAT-channel-3-pha-1-amp-1.0-Z-0-31-0.csv')
print("Voltage_RMS: {}".format( np.sqrt(np.mean(np.square(processor.voltages[0]))) ) )
print("Pressure_RMS: {}".format( pressure_rms_from_voltages_peak_to_peak( processor.voltages, 3.16/1000 )) )
print("Pressure_RMS_corrected_At_4.2: {}".format( free_field_correction(np.sqrt(np.mean(np.square(processor.voltages)))/(3.16/1000), 4.2) ) )
# print("Points: {} x {}  ".format( processor.x_size, processor.z_size ) )
print("Peak pressure: {} ".format( processor.amplitudes ) )
print("Phases: {} ".format( processor.phases ) )
print("")

processor.parse_file(f'./data/20231027-61-hex-PAT-channel-3-pha-1-amp-1.0-Z-0-31-1.csv')
print("Voltage_RMS: {}".format( np.sqrt(np.mean(np.square(processor.voltages[0]))) ) )
print("Pressure_RMS: {}".format( pressure_rms_from_voltages_peak_to_peak( processor.voltages, 3.16/1000 )) )
print("Pressure_RMS_corrected_At_4.2: {}".format( free_field_correction(np.sqrt(np.mean(np.square(processor.voltages)))/(3.16/1000), 4.2) ) )
# print("Points: {} x {}  ".format( processor.x_size, processor.z_size ) )
print("Peak pressure: {} ".format( processor.amplitudes ) )
print("Phases: {} ".format( processor.phases ) )

processor.parse_file(f'./data/20231027-61-hex-PAT-channel-3-pha-5.21-amp-1.0-0.csv')
print("Voltage_RMS: {}".format( np.sqrt(np.mean(np.square(processor.voltages[0]))) ) )
print("Pressure_RMS: {}".format( pressure_rms_from_voltages_peak_to_peak( processor.voltages, 3.16/1000 )) )
print("Pressure_RMS_corrected_At_4.2: {}".format( free_field_correction(np.sqrt(np.mean(np.square(processor.voltages)))/(3.16/1000), 4.2) ) )
# print("Points: {} x {}  ".format( processor.x_size, processor.z_size ) )
print("Peak pressure: {} ".format( processor.amplitudes ) )
print("Phases: {} ".format( processor.phases ) )




FileNotFoundError: [Errno 2] No such file or directory: './data/20231027-61-hex-PAT-channel-3-pha--1-amp-1.0-Z-0-31-0.csv'

In [89]:
processor = VADProcessor()
# processor.set_free_field_correction(7.8)
processor.set_free_field_correction(4.2)
processor.set_preamplifier_gain(3.16/1000)
processor.set_sampling_period(timebase_to_sampling_period(0)) # setting a timebase of 0 yields a 0.1 us sampling period
# processor.parse_file(f'./data/example.csv')

processor.parse_file(f'./data/20231027-61-hex-PAT-channel-3-pha-0-amp-1.0-0.csv')
print("Voltage_RMS: {}".format( np.sqrt(np.mean(np.square(processor.voltages[0]))) ) )
print("Pressure_RMS: {}".format( pressure_rms_from_voltages_peak_to_peak( processor.voltages, 3.16/1000 )) )
print("Pressure_RMS_corrected_At_4.2: {}".format( free_field_correction(np.sqrt(np.mean(np.square(processor.voltages)))/(3.16/1000), 4.2) ) )
# print("Points: {} x {}  ".format( processor.x_size, processor.z_size ) )
print("Peak pressure: {} ".format( processor.amplitudes ) )
print("Phases: {} ".format( processor.phases ) )

print(" ")

processor.parse_file(f'./data/20231027-61-hex-PAT-channel-3-pha-0-amp-1.0-1.csv')
print("Voltage_RMS: {}".format( np.sqrt(np.mean(np.square(processor.voltages[0]))) ) )
print("Pressure_RMS: {}".format( pressure_rms_from_voltages_peak_to_peak( processor.voltages, 3.16/1000 )) )
print("Pressure_RMS_corrected_At_4.2: {}".format( free_field_correction(np.sqrt(np.mean(np.square(processor.voltages)))/(3.16/1000), 4.2) ) )
# print("Points: {} x {}  ".format( processor.x_size, processor.z_size ) )
print("Peak pressure: {} ".format( processor.amplitudes ) )
print("Phases: {} ".format( processor.phases ) )
print(" ")

processor.parse_file(f'./data/20231027-61-hex-PAT-channel-3-pha-0-amp-1.0-2.csv')
print("Voltage_RMS: {}".format( np.sqrt(np.mean(np.square(processor.voltages[0]))) ) )
print("Pressure_RMS: {}".format( pressure_rms_from_voltages_peak_to_peak( processor.voltages, 3.16/1000 )) )
print("Pressure_RMS_corrected_At_4.2: {}".format( free_field_correction(np.sqrt(np.mean(np.square(processor.voltages)))/(3.16/1000), 4.2) ) )
print("Points: {} x {}  ".format( processor.x_size, processor.z_size ) )
print("Peak pressure: {} ".format( processor.amplitudes ) )
print("Phases: {} ".format( processor.phases ) )
print(" ")

processor.parse_file(f'./data/20231027-61-hex-PAT-channel-3-pha-0-amp-1.0-3.csv')
print("Voltage_RMS: {}".format( np.sqrt(np.mean(np.square(processor.voltages[0]))) ) )
print("Pressure_RMS: {}".format( pressure_rms_from_voltages_peak_to_peak( processor.voltages, 3.16/1000 )) )
print("Pressure_RMS_corrected_At_4.2: {}".format( free_field_correction(np.sqrt(np.mean(np.square(processor.voltages)))/(3.16/1000), 4.2) ) )
# print("Points: {} x {}  ".format( processor.x_size, processor.z_size ) )
print("Peak pressure: {} ".format( processor.amplitudes ) )
print("Phases: {} ".format( processor.phases ) )
print(" ")

processor.parse_file(f'./data/20231027-61-hex-PAT-channel-3-pha-0-amp-1.0-4.csv')
print("Voltage_RMS: {}".format( np.sqrt(np.mean(np.square(processor.voltages[0]))) ) )
print("Pressure_RMS: {}".format( pressure_rms_from_voltages_peak_to_peak( processor.voltages, 3.16/1000 )) )
print("Pressure_RMS_corrected_At_4.2: {}".format( free_field_correction(np.sqrt(np.mean(np.square(processor.voltages)))/(3.16/1000), 4.2) ) )
# print("Points: {} x {}  ".format( processor.x_size, processor.z_size ) )
print("Peak pressure: {} ".format( processor.amplitudes ) )
print("Phases: {} ".format( processor.phases ) )
print(" ")

processor.parse_file(f'./data/20231027-61-hex-PAT-channel-3-pha-0-amp-1.0-4.csv')
print("Voltage_RMS: {}".format( np.sqrt(np.mean(np.square(processor.voltages[0]))) ) )
print("Pressure_RMS: {}".format( pressure_rms_from_voltages_peak_to_peak( processor.voltages, 3.16/1000 )) )
print("Pressure_RMS_corrected_At_4.2: {}".format( free_field_correction(np.sqrt(np.mean(np.square(processor.voltages)))/(3.16/1000), 4.2) ) )
# print("Points: {} x {}  ".format( processor.x_size, processor.z_size ) )
print("Peak pressure: {} ".format( processor.amplitudes ) )
print("Phases: {} ".format( processor.phases ) )
print(" ")



# processor.plot()

# print("\nVoltages: {}".format( processor.voltages[0] ))
# print("#Voltages: {}".format( len(processor.voltages[0]) ))
# print("Voltage_RMS: {}".format( np.sqrt(np.mean(np.square(processor.voltages[0]))) ) )
# print("Pressure_RMS: {}".format( pressure_rms_from_voltages_peak_to_peak( processor.voltages, 3.16/1000 )) )
# print("Pressure_RMS_corrected_At_4.2: {}".format( free_field_correction(np.sqrt(np.mean(np.square(processor.voltages)))/(3.16/1000), 4.2) ) )
# print("Points: {} x {}  ".format( processor.x_size, processor.z_size ) )
# print("Peak pressure: {} ".format( processor.amplitudes ) )
# print("Phases: {} ".format( processor.phases ) )
# print("Coords:{} ".format(  np.squeeze(( np.where(processor.amplitudes == np.amax(processor.amplitudes)) ) ) ) )


Voltage_RMS: 7.834272589128921
Pressure_RMS: 2205.3182381323504
Pressure_RMS_corrected_At_4.2: 1359.7882031463803
Peak pressure: [[1528.6624435687734, 1166.7214956683003]] 
Phases: [[0.5026548245743663, 1.231504320207197]] 
 
Voltage_RMS: 7.831461079596763
Pressure_RMS: 2205.8861255322026
Pressure_RMS_corrected_At_4.2: 1360.1383596787446
Peak pressure: [[1528.1138477696468, 1168.2554443244958]] 
Phases: [[0.47752208334564816, 1.2063715789784792]] 
 
Voltage_RMS: 7.830141950430871
Pressure_RMS: 2205.408625960276
Pressure_RMS_corrected_At_4.2: 1359.8439358293056
Points: 1 x 1  
Peak pressure: [[1527.8564526903806, 1167.9065543081128]] 
Phases: [[0.47752208334564816, 1.2315043202071978]] 
 
Voltage_RMS: 7.830891786882951
Pressure_RMS: 2206.495052026486
Pressure_RMS_corrected_At_4.2: 1360.5138207116227
Peak pressure: [[1528.0027645285343, 1169.27467444046]] 
Phases: [[0.47752208334564816, 1.2063715789784801]] 
 
Voltage_RMS: 7.830272410980167
Pressure_RMS: 2205.2999074405934
Pressure_RMS_c

In [103]:
processor = VADProcessor()
# processor.set_free_field_correction(7.8)
processor.set_free_field_correction(4.2)
processor.set_preamplifier_gain(3.16/1000)
processor.set_sampling_period(timebase_to_sampling_period(0)) # setting a timebase of 0 yields a 0.1 us sampling period
# processor.parse_file(f'./data/example.csv')
print(" ")

processor.parse_file(f'./data/20231027-61-hex-PAT-channel-3-pha-3.142-amp-1.0-0.csv')
print("Voltage_RMS: {}".format( np.sqrt(np.mean(np.square(processor.voltages[0]))) ) )
print("Pressure_RMS: {}".format( pressure_rms_from_voltages_peak_to_peak( processor.voltages, 3.16/1000 )) )
print("Pressure_RMS_corrected_At_4.2: {}".format( free_field_correction(np.sqrt(np.mean(np.square(processor.voltages)))/(3.16/1000), 4.2) ) )
# print("Points: {} x {}  ".format( processor.x_size, processor.z_size ) )
print("Peak pressure: {} ".format( processor.amplitudes ) )
print("Phases: {} ".format( processor.phases ) )
print(" ")

processor.parse_file(f'./data/20231027-61-hex-PAT-channel-3-pha-3.142-amp-1.0-1.csv')
print("Voltage_RMS: {}".format( np.sqrt(np.mean(np.square(processor.voltages[0]))) ) )
print("Pressure_RMS: {}".format( pressure_rms_from_voltages_peak_to_peak( processor.voltages, 3.16/1000 )) )
print("Pressure_RMS_corrected_At_4.2: {}".format( free_field_correction(np.sqrt(np.mean(np.square(processor.voltages)))/(3.16/1000), 4.2) ) )
# print("Points: {} x {}  ".format( processor.x_size, processor.z_size ) )
print("Peak pressure: {} ".format( processor.amplitudes ) )
print("Phases: {} ".format( processor.phases ) )
print(" ")

processor.parse_file(f'./data/20231027-61-hex-PAT-channel-3-pha-6.28-amp-1.0-0.csv')
print("Voltage_RMS: {}".format( np.sqrt(np.mean(np.square(processor.voltages[0]))) ) )
print("Pressure_RMS: {}".format( pressure_rms_from_voltages_peak_to_peak( processor.voltages, 3.16/1000 )) )
print("Pressure_RMS_corrected_At_4.2: {}".format( free_field_correction(np.sqrt(np.mean(np.square(processor.voltages)))/(3.16/1000), 4.2) ) )
# print("Points: {} x {}  ".format( processor.x_size, processor.z_size ) )
print("Peak pressure: {} ".format( processor.amplitudes ) )
print("Phases: {} ".format( processor.phases ) )
print(" ")

processor.parse_file(f'./data/20231027-61-hex-PAT-channel-3-pha-6.28-amp-1.0-1.csv')
print("Voltage_RMS: {}".format( np.sqrt(np.mean(np.square(processor.voltages[0]))) ) )
print("Pressure_RMS: {}".format( pressure_rms_from_voltages_peak_to_peak( processor.voltages, 3.16/1000 )) )
print("Pressure_RMS_corrected_At_4.2: {}".format( free_field_correction(np.sqrt(np.mean(np.square(processor.voltages)))/(3.16/1000), 4.2) ) )
# print("Points: {} x {}  ".format( processor.x_size, processor.z_size ) )
print("Peak pressure: {} ".format( processor.amplitudes ) )
print("Phases: {} ".format( processor.phases ) )
print(" ")


 
Voltage_RMS: 4.251612565968712
Pressure_RMS: 1255.3770045035938
Pressure_RMS_corrected_At_4.2: 774.059186428755
Peak pressure: [[829.5959044391707, 714.2169723041104]] 
Phases: [[2.8148670176164527, 3.0913271711323542]] 
 
Voltage_RMS: 4.248591330504199
Pressure_RMS: 1254.2814523883392
Pressure_RMS_corrected_At_4.2: 773.3836744702106
Peak pressure: [[829.00638586738, 713.4374731293553]] 
Phases: [[2.8148670176164536, 3.0913271711323542]] 
 
Voltage_RMS: 4.261453335570011
Pressure_RMS: 1255.2172762144617
Pressure_RMS_corrected_At_4.2: 773.9606987640223
Peak pressure: [[831.5160846134236, 711.7663433260578]] 
Phases: [[-1.9603538158400327, -1.6587609210954124]] 
 
Voltage_RMS: 4.251016913552041
Pressure_RMS: 1251.1956808914947
Pressure_RMS_corrected_At_4.2: 771.4810031883693
Peak pressure: [[829.4796777610164, 708.7519599567598]] 
Phases: [[-1.9603538158400333, -1.6838936623241307]] 
 


In [ ]:
# processor.parse_file(f'./data/61-hex-PAT-channel-3-pha-0-amp-1.0-4.csv')

# Peak pressure: [[1510.983524007819, 1007.4934376744214]] 
# Phases: [[-2.060884780754905, -1.2063715789784815]] 

# Peak pressure: [[1500.271844706212, 997.8235086156312]] 
# Phases: [[-1.3069025438933541, -0.4775220833456495]] 

# Peak pressure: [[1482.2446156743079, 991.7434781377444]] 
# Phases: [[1.4576989912656613, 2.236813969355932]] 

# Peak pressure: [[1481.6258033745419, 989.5157392058973]] 
# Phases: [[-2.31221219304209, -1.4828317324943832]] 

# Peak pressure: [[1476.8485940326718, 989.1251909757029]] 
# Phases: [[-1.8598228509251578, -1.0555751316061728]] 


# processor.parse_file(f'./data/61-hex-PAT-channel-3-pha-0-amp-0.5-4.csv')

# Peak pressure: [[827.8050855635198, 578.4892839860172]] 
# Phases: [[1.3069025438933517, 2.2870794518133666]] 

# Peak pressure: [[838.2638111404383, 578.9444632180479]] 
# Phases: [[1.3823007675795074, 2.312212193042086]] 

# Peak pressure: [[827.3526640716087, 576.8696105203735]] 
# Phases: [[-2.7897342763877386, -1.8346901096964392]]

# Peak pressure: [[827.215686337523, 576.8813691387459]] 
# Phases: [[-1.1812388377497625, -0.251327412287186]] 

# Peak pressure: [[826.0639317025722, 576.4586232938028]] 
# Phases: [[2.1111502632123393, 3.0913271711323542]]

In [49]:
# Voltage_RMS: 7.0615709603505445
# Pressure_RMS: 1921.3256291413632
# Pressure_RMS_corrected_At_4.2: 1184.6797798769337
# Points: 1 x 1  
# Peak pressure: [[1377.8890377982077, 953.0761570903171]] Pa 
# Phases: [[-0.3015928947446209, 0.3518583772020554]] 
# Coords:[0 0] 

# Voltage_RMS: 7.030503164412646
# Pressure_RMS: 1914.7928440976707
# Pressure_RMS_corrected_At_4.2: 1180.6516972707582
# Points: 1 x 1  
# Peak pressure: [[1371.8269340975241, 951.8233677638013]] Pa 
# Phases: [[2.2368139693559312, 2.915397982531326]] 
# Coords:[0 0] 

# Voltage_RMS: 7.0302567560927285
# Pressure_RMS: 1915.0010453828722
# Pressure_RMS_corrected_At_4.2: 1180.7800731425943
# Points: 1 x 1  
# Peak pressure: [[1371.7788536739547, 952.21107894953]] Pa 
# Phases: [[-3.0410616886749198, -2.337344934270807]] 
# Coords:[0 0] 

# Voltage_RMS: 7.049053035600589
# Pressure_RMS: 1917.2922216035943
# Pressure_RMS_corrected_At_4.2: 1182.1928009486737
# Points: 1 x 1  
# Peak pressure: [[1375.446477724, 950.4244442071093]] Pa 
# Phases: [[-1.4576989912656648, -0.7791149780902695]] 
# Coords:[0 0] 

# Voltage_RMS: 7.023450127945123
# Pressure_RMS: 1912.8970266658096
# Pressure_RMS_corrected_At_4.2: 1179.48274571783
# Points: 1 x 1  
# Peak pressure: [[1370.4507103526432, 950.9069067927072]] Pa 
# Phases: [[-0.2513274122871856, 0.40212385965949293]] 
# Coords:[0 0] 


# Voltage_RMS: 4.1528549165870405
# Pressure_RMS: 1132.307565932713
# Pressure_RMS_corrected_At_4.2: 698.1751857240524
# Points: 1 x 1  
# Peak pressure: [[810.3258180453965, 564.1535682158616]] Pa 
# Phases: [[0.7791149780902664, 1.8346901096964379]] 
# Coords:[0 0] 

# Voltage_RMS: 4.030285623282947
# Pressure_RMS: 1110.6249607959041
# Pressure_RMS_corrected_At_4.2: 684.8057997693602
# Points: 1 x 1  
# Peak pressure: [[786.4094846172325, 565.2239284562918]] Pa 
# Phases: [[1.6587609210954106, 2.513274122871832]] 
# Coords:[0 0] 

# Voltage_RMS: 4.026659904708518
# Pressure_RMS: 1108.97312238962
# Pressure_RMS_corrected_At_4.2: 683.7872844641629
# Points: 1 x 1  
# Peak pressure: [[785.702016278263, 563.739693835746]] Pa 
# Phases: [[-0.9047786842338628, -1.7763568394002505e-15]] 
# Coords:[0 0] 

# Voltage_RMS: 4.02769306852555
# Pressure_RMS: 1110.346929569794
# Pressure_RMS_corrected_At_4.2: 684.6343671049788
# Points: 1 x 1  
# Peak pressure: [[785.9036123686694, 565.5121088963059]] Pa 
# Phases: [[2.437875899185676, -2.9656634649887654]] 
# Coords:[0 0] 

# Voltage_RMS: 4.036553347645788
# Pressure_RMS: 1113.71820358442
# Pressure_RMS_corrected_At_4.2: 686.713077812302
# Points: 1 x 1  
# Peak pressure: [[787.6324743372252, 568.1415209666627]] Pa 
# Phases: [[0.35185837720205493, 1.2566370614359155]] 
# Coords:[0 0] 

In [114]:
processor = VADProcessor()
# processor.set_free_field_correction(7.8)
processor.set_free_field_correction(4.2)
processor.set_preamplifier_gain(3.16/1000)
processor.set_sampling_period(timebase_to_sampling_period(0)) # setting a timebase of 0 yields a 0.1 us sampling period
# processor.parse_file(f'./data/example.csv')
print(" ")

processor.parse_file(f'./data/20231027-61-hex-PAT-channel-3-pha-5.31-amp-1.0-0.csv')
print("Voltage_RMS: {}".format( np.sqrt(np.mean(np.square(processor.voltages[0]))) ) )
print("Pressure_RMS: {}".format( pressure_rms_from_voltages_peak_to_peak( processor.voltages, 3.16/1000 )) )
print("Pressure_RMS_corrected_At_4.2: {}".format( free_field_correction(np.sqrt(np.mean(np.square(processor.voltages)))/(3.16/1000), 4.2) ) )
# print("Points: {} x {}  ".format( processor.x_size, processor.z_size ) )
print("Peak pressure: {} ".format( processor.amplitudes ) )
print("Phases: {} ".format( processor.phases ) )
print(" ")


processor.parse_file(f'./data/20231027-61-hex-PAT-channel-3-pha-5.21-amp-1.0-0.csv')
print("Voltage_RMS: {}".format( np.sqrt(np.mean(np.square(processor.voltages[0]))) ) )
print("Pressure_RMS: {}".format( pressure_rms_from_voltages_peak_to_peak( processor.voltages, 3.16/1000 )) )
print("Pressure_RMS_corrected_At_4.2: {}".format( free_field_correction(np.sqrt(np.mean(np.square(processor.voltages)))/(3.16/1000), 4.2) ) )
# print("Points: {} x {}  ".format( processor.x_size, processor.z_size ) )
print("Peak pressure: {} ".format( processor.amplitudes ) )
print("Phases: {} ".format( processor.phases ) )
print(" ")


processor.parse_file(f'./data/20231027-61-hex-PAT-channel-3-pha-5.236-amp-1.0-0.csv')
print("Voltage_RMS: {}".format( np.sqrt(np.mean(np.square(processor.voltages[0]))) ) )
print("Pressure_RMS: {}".format( pressure_rms_from_voltages_peak_to_peak( processor.voltages, 3.16/1000 )) )
print("Pressure_RMS_corrected_At_4.2: {}".format( free_field_correction(np.sqrt(np.mean(np.square(processor.voltages)))/(3.16/1000), 4.2) ) )
# print("Points: {} x {}  ".format( processor.x_size, processor.z_size ) )
print("Peak pressure: {} ".format( processor.amplitudes ) )
print("Phases: {} ".format( processor.phases ) )
print(" ")


processor.parse_file(f'./data/20231027-61-hex-PAT-channel-3-pha-5.497-amp-1.0-0.csv')
print("Voltage_RMS: {}".format( np.sqrt(np.mean(np.square(processor.voltages[0]))) ) )
print("Pressure_RMS: {}".format( pressure_rms_from_voltages_peak_to_peak( processor.voltages, 3.16/1000 )) )
print("Pressure_RMS_corrected_At_4.2: {}".format( free_field_correction(np.sqrt(np.mean(np.square(processor.voltages)))/(3.16/1000), 4.2) ) )
# print("Points: {} x {}  ".format( processor.x_size, processor.z_size ) )
print("Peak pressure: {} ".format( processor.amplitudes ) )
print("Phases: {} ".format( processor.phases ) )
print(" ")

processor.parse_file(f'./data/20231027-61-hex-PAT-channel-3-pha-3.142-amp-1.0-1.csv')
print("Voltage_RMS: {}".format( np.sqrt(np.mean(np.square(processor.voltages[0]))) ) )
print("Pressure_RMS: {}".format( pressure_rms_from_voltages_peak_to_peak( processor.voltages, 3.16/1000 )) )
print("Pressure_RMS_corrected_At_4.2: {}".format( free_field_correction(np.sqrt(np.mean(np.square(processor.voltages)))/(3.16/1000), 4.2) ) )
# print("Points: {} x {}  ".format( processor.x_size, processor.z_size ) )
print("Peak pressure: {} ".format( processor.amplitudes ) )
print("Phases: {} ".format( processor.phases ) )
print(" ")

processor.parse_file(f'./data/20231027-61-hex-PAT-channel-3-pha-4.712-amp-1.0-0.csv')
print("Voltage_RMS: {}".format( np.sqrt(np.mean(np.square(processor.voltages[0]))) ) )
print("Pressure_RMS: {}".format( pressure_rms_from_voltages_peak_to_peak( processor.voltages, 3.16/1000 )) )
print("Pressure_RMS_corrected_At_4.2: {}".format( free_field_correction(np.sqrt(np.mean(np.square(processor.voltages)))/(3.16/1000), 4.2) ) )
# print("Points: {} x {}  ".format( processor.x_size, processor.z_size ) )
print("Peak pressure: {} ".format( processor.amplitudes ) )
print("Phases: {} ".format( processor.phases ) )
print(" ")

processor.parse_file(f'./data/20231027-61-hex-PAT-channel-3-pha-1.57-amp-1.0-0.csv')
print("Voltage_RMS: {}".format( np.sqrt(np.mean(np.square(processor.voltages[0]))) ) )
print("Pressure_RMS: {}".format( pressure_rms_from_voltages_peak_to_peak( processor.voltages, 3.16/1000 )) )
print("Pressure_RMS_corrected_At_4.2: {}".format( free_field_correction(np.sqrt(np.mean(np.square(processor.voltages)))/(3.16/1000), 4.2) ) )
# print("Points: {} x {}  ".format( processor.x_size, processor.z_size ) )
print("Peak pressure: {} ".format( processor.amplitudes ) )
print("Phases: {} ".format( processor.phases ) )
print(" ")

processor.parse_file(f'./data/20231027-61-hex-PAT-channel-3-pha-5.27-amp-1.0-0.csv')
print("Voltage_RMS: {}".format( np.sqrt(np.mean(np.square(processor.voltages[0]))) ) )
print("Pressure_RMS: {}".format( pressure_rms_from_voltages_peak_to_peak( processor.voltages, 3.16/1000 )) )
print("Pressure_RMS_corrected_At_4.2: {}".format( free_field_correction(np.sqrt(np.mean(np.square(processor.voltages)))/(3.16/1000), 4.2) ) )
# print("Points: {} x {}  ".format( processor.x_size, processor.z_size ) )
print("Peak pressure: {} ".format( processor.amplitudes ) )
print("Phases: {} ".format( processor.phases ) )
print(" ")



# Voltage_RMS: 6.935343666681421
# Pressure_RMS: 1936.8047714435654
# Pressure_RMS_corrected_At_4.2: 1194.224141653573
# Peak pressure: [[1353.2589370466656, 1010.4617015601168]] 
# Phases: [[1.0555751316061706, 1.2566370614359164]] 

 
Voltage_RMS: 6.849101720399521
Pressure_RMS: 1922.6928455869668
Pressure_RMS_corrected_At_4.2: 1185.5227987037538
Peak pressure: [[1336.4309772276047, 1012.3639936303769]] 
Phases: [[0.07539822368615479, 0.22619467105846436]] 
 
Voltage_RMS: 6.873893700632895
Pressure_RMS: 1934.453031719207
Pressure_RMS_corrected_At_4.2: 1192.7740706938564
Peak pressure: [[1341.2685123852445, 1022.9461105664634]] 
Phases: [[-0.050265482457437116, 0.15079644737231002]] 
 
Voltage_RMS: 6.869819526360582
Pressure_RMS: 1932.7048436434652
Pressure_RMS_corrected_At_4.2: 1191.6961466640378
Peak pressure: [[1340.473539127964, 1021.4744748869664]] 
Phases: [[-0.025132741228719002, 0.15079644737230913]] 
 
Voltage_RMS: 6.877891166893472
Pressure_RMS: 1932.6735933018567
Pressure_RMS_corrected_At_4.2: 1191.6768778595972
Peak pressure: [[1342.048517991694, 1019.3592780794639]] 
Phases: [[0.20106192982974536, 0.40212385965949116]] 
 
Voltage_RMS: 6.879278549359624
Pressure_RMS: 1930.3337626337855
Pressure_RMS_corr